In [ ]:
// boring imports
var Plot = require('plotly-notebook-js');
var {loadLabelledWine} = require('./utils');

# Supervised Learning


![Supervised learning](images/slide_supervised.png)

### Techniques availble in mljs

 - KNN - K Nearest Neighbors
 - **SVM - Support Vector Machines**
 - Naive Bayes
 - Partial Least Squares [regression]



In [ ]:
var {features, dataset} = loadLabelledWine({verbose: true});

## Support Vector Machines


##### SVM in mljs

 - docs for the SVM module are [here](https://mljs.github.io/svm/)
 - docs for the various kernel options are [here](https://github.com/mljs/kernel)


In [ ]:
var SVM = require('ml-svm');

var options = {
  C: 1,  // Regularisation Parameter
  tol: 10e-4,  // numerical tolerance
  maxPasses: 10,  // max number of times to iterate over alphas when no change
  maxIterations: 10000,  // max number of iterations
  kernel: 'linear', // kernel to use see full like in kernel docs
  kernelOptions: {
    sigma: 0.5  // sets the gaussian width
  }
};

var svm = new SVM(options);

Next train the classifier, let's start with the original 2d feature vectors that we used in our first pass through kmeans.

To do this, just call the `svm.train(inputs, labels)` with the input feature vectors and the true class labels.

In [ ]:
// prep our input data structures
var inputs = dataset.map(d => [d[0], d[11]]);
var labels = dataset.map(d => d[13] - 2); // needs label domain of {-1,1} rather then our labels of [0,1,2]

// train the model
svm.train(inputs, labels)

console.log(labels.slice(50,60))
console.log(inputs.slice(50,60).map(i => svm.predict(i)))

#### Margins

In [ ]:
console.log(svm.margin(inputs.slice(0,5)))

#### Support Vectors

In [ ]:
console.log(svm.supportVectors().slice(0,5))

#### Measure Accuracy

Use the same confusion matrix approach as earlier to compute accuracy and f1-scores

In [ ]:
var ConfusionMatrix = require('ml-confusion-matrix');

// finish this off

#### Discussion

 - How does you accuracy compare with our unsupervised approaches?
 - Is this too good to be tru and should you be suspicous?
 - Can you think of how to get a better accuracy measurement?